In [114]:
import numpy as np
# import cv2
import tifffile as tiff
from shapely.geometry import Polygon, mapping, shape, box
from skimage import io
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
# import json
from rasterio import features
import scanpy as sc
import pandas as pd
import geopandas as gpd

from copy import copy

import pickle
import scipy

import seaborn as sns

# from SGanalysis.SGobject import SGobject

import matplotlib
# %matplotlib inline
matplotlib.use('Qt5Agg')
%matplotlib qt






In [115]:
%load_ext autoreload
%autoreload 2

# from ../../software/SGanalysis.SGobject import SGobject

import sys
import os

# software_dir = '~/Documents/Penn/Research/software/'
# software_dir = os.path.expanduser(software_dir)

# sganalysis_path = f'{software_dir}/SGanalysis/SGobject.py'

sganalysis_path = '/Users/grantkinsler/Documents/Penn/Research/software/SGanalysis/SGanalysis/SGobject.py'
sys.path.append(os.path.dirname(os.path.expanduser(sganalysis_path)))
from SGobject import SGobject
# import figures as figures

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# filepath  = '/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/Shared_Grant/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/'
filepath = '/Users/grantkinsler/RajLab Dropbox/Grant Kinsler/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/'

# roi_2
roi_name = 'roi_2'

In [105]:
## import segmentation

# from SGanalysis.SGobject import SGobject

# Create an instance of SGobject
sg_obj = SGobject()

# Convert a TIFF image to polygons and store them in a GeoDataFrame
print("Running mask_to_objects...")
sg_obj.mask_to_objects(f'{filepath}{roi_name}/segmentations/segmentation_1_nuclei.tiff')

Running mask_to_objects...


In [106]:
## import spots

spot_file = f'{filepath}{roi_name}/exports/decode_20240314_withcoordinates.csv'

sg_obj.load_points(spot_file)

In [107]:
sg_obj.create_cell_gene_table()

5337507 of 12227072 spots (43.7%) assigned to an object.


/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/anndata/utils.py:143: UserWarning: X converted to numpy array with dtype object
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/SGanalysis/SGobject.py:217: FutureWarning: X.dtype being converted to np.float32 from object. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  self.cell_gene_table = sc.AnnData(pivot_table_full)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [4]:
# with open(f'{filepath}{roi_name}/exports/sg_object_decode_20240314_segmentation_1_20240521.pkl', 'wb') as f:
#     pickle.dump(sg_obj, f)

with open(f'{filepath}{roi_name}/exports/sg_object_decode_20240314_segmentation_1.pkl', 'rb') as f:
    sg_obj = pickle.load(f)

In [5]:
# sg_obj_only_bcs = sg_obj.copy()

sg_obj_only_bcs = copy(sg_obj)

bc_list = np.unique([name for name in sg_obj.points_gdf['name'].values if 'bc_' in name])


# filter the points gdf to only include the barcodes 
sg_obj_only_bcs.points_gdf = sg_obj.points_gdf.loc[sg_obj.points_gdf['name'].isin(bc_list)]

In [110]:
# run proseg stuff (note that this will essentially just make the csv file; I'm running the rest manually)

sg_obj_only_bcs.run_proseg(output_path='proseg/roi2/roi2_proseg_barcodesonly.csv')

OSError: Cannot save file into a non-existent directory: 'proseg/roi2'

In [32]:



sg_obj_only_bcs.gdf = sg_obj_only_bcs._load_and_process_geometries('proseg/onlybarcodes/roi2/cell-polygons.geojson')

Total MultiPolygons processed into largest Polygons: 5526
Total Polygons simplified: 10103
cell column exists


In [33]:
sg_obj_only_bcs.create_cell_gene_table()

2132806 of 2308158 spots (92.4%) assigned to an object.


/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/anndata/utils.py:143: UserWarning: X converted to numpy array with dtype object
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/SGanalysis/SGobject.py:217: FutureWarning: X.dtype being converted to np.float32 from object. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  self.cell_gene_table = sc.AnnData(pivot_table_full)
/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [49]:
proseg_out = pd.read_csv('roi2_proseg_barcodesonly.csv')

In [34]:
sg_obj_only_bcs.get_cell_gene_table_df()

No filtering available or applied.


,object_id,bc_001,bc_002,bc_003,bc_004,bc_005,bc_006,bc_007,bc_008,bc_009,...,bc_087,bc_088,bc_089,bc_090,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096
0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,3.0,0.0,...,87.0,19.0,0.0,0.0,4.0,73.0,0.0,0.0,1.0,1.0
1,1.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4.0,0.0,12.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15624,15624.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
15625,15625.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15626,15626.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0
15627,15627.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
sg_obj_only_bcs.get_cell_gene_table_df()

No filtering available or applied.


,object_id,bc_001,bc_002,bc_003,bc_004,bc_005,bc_006,bc_007,bc_008,bc_009,...,bc_087,bc_088,bc_089,bc_090,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,4.0,2.0,0.0,2.0
2,2.0,0.0,0.0,0.0,0.0,13.0,0.0,1.0,0.0,2.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15624,15624.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,2.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
15625,15625.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15626,15626.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15627,15627.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:

df = sg_obj_only_bcs.get_cell_gene_table_df()

barcode_cols = ['bc_{:03d}'.format(i) for i in range(1,97)]
gene_cols = [col for col in df.columns if 'bc_' not in col and col not in ['cell_id','called_barcodes','n_called_barcodes','barcode_names']]

No filtering available or applied.


In [26]:
for bc in ['bc_{:03d}'.format(i) for i in range(1,97,10)]:

    vals = [len(df[df[bc]>c][bc].values) for c in range(50)]
    plt.plot(range(50),vals,label=bc,marker='o')
plt.legend()
plt.xlabel('number of barcode spots cutoff')
plt.ylabel('number of cells with at least cutoff barcode spots')

Text(0, 0.5, 'number of cells with at least cutoff barcode spots')

In [27]:
# find number of cells with at least cutoff spots per barcode

# cutoff = 1
cutoff = 5
# cutoff = 3

n_cells = {}

for bc in ['bc_{:03d}'.format(i) for i in range(1,97)]:
    n_cells[bc] = len(df[df[bc]>cutoff][bc].values)


In [28]:
cell_barcodes = {}

# df.sort_index(inplace=True)

df['cell_id'] = df.index

for cell_id in df['cell_id']:
    this_cell = df[df['cell_id']==cell_id]
    cell_barcodes[cell_id] = []

    for bc in barcode_cols:
        if this_cell[bc].values[0] > cutoff:
            cell_barcodes[cell_id].append(bc)

df['called_barcodes'] = cell_barcodes.values()
df['n_called_barcodes'] = [len(bc_set) for bc_set in cell_barcodes.values()]
df['barcode_names'] = ['-'.join(sorted(bc_set)) for bc_set in cell_barcodes.values()]

df = pd.merge(df,sg_obj_only_bcs.gdf[['nucleus_centroid','nucleus']],how='left',left_index=True,right_index=True)

# # get each cell polygon's position
# df['location'] = sg_obj.gdf['nucleus'].centroid.values
# df['location_x'] = sg_obj.gdf['nucleus'].centroid.x.values
# df['location_y'] = sg_obj.gdf['nucleus'].centroid.y.values

KeyError: "['nucleus_centroid'] not in index"

In [85]:
df

name,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096,cell_id,called_barcodes,n_called_barcodes,barcode_names
object_id,,,,,,,,,,,,,,,,,,,,,
1,16.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,74.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,[],0,
10,11.0,0.0,2.0,0.0,5.0,0.0,0.0,10.0,28.0,2.0,...,3.0,0.0,0.0,0.0,0.0,0.0,10,"[bc_019, bc_054, bc_060, bc_076]",4,bc_019-bc_054-bc_060-bc_076
100,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,100,[],0,
1000,3.0,0.0,0.0,0.0,5.0,0.0,0.0,15.0,30.0,4.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1000,"[bc_049, bc_060, bc_066, bc_078, bc_081, bc_084]",6,bc_049-bc_060-bc_066-bc_078-bc_081-bc_084
10000,5.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,33.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10000,"[bc_001, bc_008]",2,bc_001-bc_008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,14.0,0.0,0.0,0.0,3.0,0.0,0.0,10.0,100.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,9995,[bc_022],1,bc_022
9996,2.0,0.0,38.0,0.0,0.0,0.0,0.0,17.0,19.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9996,[bc_060],1,bc_060
9997,5.0,0.0,0.0,0.0,5.0,0.0,0.0,13.0,25.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9997,[bc_073],1,bc_073


In [9]:
sg_obj_only_bcs.get_cell_gene_table_df()

No filtering available or applied.


name,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_087,bc_088,bc_089,bc_090,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096
object_id,,,,,,,,,,,,,,,,,,,,,
1,16.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,74.0,1.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
10,11.0,0.0,2.0,0.0,5.0,0.0,0.0,10.0,28.0,2.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,3.0,0.0,0.0,0.0,5.0,0.0,0.0,15.0,30.0,4.0,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
10000,5.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,33.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,14.0,0.0,0.0,0.0,3.0,0.0,0.0,10.0,100.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9996,2.0,0.0,38.0,0.0,0.0,0.0,0.0,17.0,19.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,5.0,0.0,0.0,0.0,5.0,0.0,0.0,13.0,25.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
sg_obj.plot_polygon_and_points('11',gene_names=['bc_001','bc011','bc012'])

/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/geopandas/plotting.py:402: UserWarning: The GeoSeries you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(


In [11]:
sg_obj_only_bcs.assigned_points_gdf['name']

,name,refid,x,y,geometry,index_right,object_id,distance_to_edge
12224769,L1CAM,ENSG00000198910,810,34215,POINT (810.000 34215.000),15719.0,15559,171.058470
10917539,SPARC,ENSG00000113140,811,34205,POINT (811.000 34205.000),15719.0,15559,170.988304
12224738,L1CAM,ENSG00000198910,814,34167,POINT (814.000 34167.000),15719.0,15559,170.586899
6323497,bc_089,bc_089,806,34220,POINT (806.000 34220.000),15719.0,15559,167.725967
12224752,L1CAM,ENSG00000198910,817,34234,POINT (817.000 34234.000),15719.0,15559,167.451486
...,...,...,...,...,...,...,...,...
12264793,ENSMUSG00000071361,ENSMUSG00000071361,16017,21280,POINT (16017.000 21280.000),NaN,NaN,NaN
12264800,ENSMUSG00000071361,ENSMUSG00000071361,2459,25574,POINT (2459.000 25574.000),NaN,NaN,NaN
12264802,ENSMUSG00000071361,ENSMUSG00000071361,15119,26361,POINT (15119.000 26361.000),NaN,NaN,NaN
12264812,ENSMUSG00000071361,ENSMUSG00000071361,8627,33611,POINT (8627.000 33611.000),NaN,NaN,NaN


No filtering available or applied.


name,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_087,bc_088,bc_089,bc_090,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096
object_id,,,,,,,,,,,,,,,,,,,,,
1,16.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,74.0,1.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
10,11.0,0.0,2.0,0.0,5.0,0.0,0.0,10.0,28.0,2.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,3.0,0.0,0.0,0.0,5.0,0.0,0.0,15.0,30.0,4.0,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
10000,5.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,33.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,14.0,0.0,0.0,0.0,3.0,0.0,0.0,10.0,100.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9996,2.0,0.0,38.0,0.0,0.0,0.0,0.0,17.0,19.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,5.0,0.0,0.0,0.0,5.0,0.0,0.0,13.0,25.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
sg_obj_only_bcs.plot_polygon_and_points('11',gene_names=['bc_006', 'bc_053', 'bc_074', 'bc_076'])

# polygon_gdf = sg_obj.gdf[sg_obj.gdf['object_id'] == '11']

# polygon_gdf.plot(ax=plt.gca(), color='blue', edgecolor='black',alpha=0.5)
# sg_obj.plot_polygon_and_points('11',gene_names=[])

/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/geopandas/plotting.py:402: UserWarning: The GeoSeries you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(


In [18]:
sg_obj.plot_polygon_and_points('11',gene_names=['bc_006', 'bc_053', 'bc_074', 'bc_076'])


: 

In [39]:

sg_obj.plot_polygon_and_points('10',gene_names=['bc_019', 'bc_054', 'bc_060', 'bc_076'],annotate=False)

In [37]:

sg_obj.plot_polygon_and_points('10',gene_names=['bc_019', 'bc_054', 'bc_060', 'bc_076'])


In [52]:
sg_obj_only_bcs.plot_polygon_and_points('10822',gene_names=['bc_019', 'bc_054', 'bc_060', 'bc_076'])


In [ ]:
sg_obj_only_bcs.plot_polygon_and_points('10',gene_names=['bc_019', 'bc_054', 'bc_060', 'bc_076'])

In [14]:
sg_obj_only_bcs.plot_polygon_and_points('1',gene_names=['AKT1', 'CAPG'],annotate=False)

/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/geopandas/plotting.py:402: UserWarning: The GeoSeries you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(


In [19]:
# sg_obj.get_cell_gene_table_df()[sg_obj.get_cell_gene_table_df['object_id'] == '1']

In [40]:
proseg_df = sg_obj_only_bcs.get_cell_gene_table_df()

No filtering available or applied.


In [66]:
proseg_df[(proseg_df['bc_006'] > 2) & (proseg_df['bc_053'] > 2) & (proseg_df['bc_074'] > 2) & (proseg_df['bc_076'] > 2)]

,object_id,bc_001,bc_002,bc_003,bc_004,bc_005,bc_006,bc_007,bc_008,bc_009,...,bc_087,bc_088,bc_089,bc_090,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096
3466,3466.0,0.0,13.0,5.0,1.0,5.0,16.0,12.0,0.0,0.0,...,133.0,5.0,3.0,0.0,115.0,22.0,1.0,35.0,26.0,2.0
3608,3608.0,77.0,0.0,0.0,1.0,0.0,3.0,1.0,0.0,1.0,...,0.0,6.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
6084,6084.0,0.0,0.0,0.0,5.0,0.0,35.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10822,10822.0,5.0,69.0,4.0,3.0,2.0,31.0,19.0,1.0,4.0,...,518.0,6.0,5.0,1.0,354.0,145.0,15.0,146.0,110.0,1.0


In [47]:
df = sg_obj.get_cell_gene_table_df()
df[(df['bc_006'] > 1) & (df['bc_053'] > 1) & (df['bc_074'] > 1) & (df['bc_076'] > 1)]

No filtering available or applied.


name,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_087,bc_088,bc_089,bc_090,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096
object_id,,,,,,,,,,,,,,,,,,,,,


In [51]:
cell_barcodes = {}

# df.sort_index(inplace=True)

df['cell_id'] = df.index

cutoff = 2 

for cell_id in df['cell_id']:
    this_cell = df[df['cell_id']==cell_id]
    cell_barcodes[cell_id] = []

    for bc in barcode_cols:
        if this_cell[bc].values[0] > cutoff:
            cell_barcodes[cell_id].append(bc)

df['called_barcodes'] = cell_barcodes.values()
df['n_called_barcodes'] = [len(bc_set) for bc_set in cell_barcodes.values()]
df['barcode_names'] = ['-'.join(sorted(bc_set)) for bc_set in cell_barcodes.values()]

# df = pd.merge(df,sg_obj.gdf[['nucleus_centroid','nucleus']],how='left',left_index=True,right_index=True)

# # get each cell polygon's position
# df['location'] = sg_obj.gdf['nucleus'].centroid.values
# df['location_x'] = sg_obj.gdf['nucleus'].centroid.x.values
# df['location_y'] = sg_obj.gdf['nucleus'].centroid.y.values

In [79]:
join_gdf = sg_obj.gdf.sjoin(sg_obj_only_bcs.gdf)

In [81]:
# get gdf mapping dictionary
mapping_dict = {}

for i in range(len(join_gdf)):
    mapping_dict[join_gdf.iloc[i]['object_id_left']] = join_gdf.iloc[i]['object_id_right']   

In [95]:
# df.sort_index(inplace=True) 
df.sort_values('cell_id',inplace=True)

df[df['n_called_barcodes']>2]

name,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096,cell_id,called_barcodes,n_called_barcodes,barcode_names
object_id,,,,,,,,,,,,,,,,,,,,,
10,11.0,0.0,2.0,0.0,5.0,0.0,0.0,10.0,28.0,2.0,...,3.0,0.0,0.0,0.0,0.0,0.0,10,"[bc_019, bc_054, bc_060, bc_076]",4,bc_019-bc_054-bc_060-bc_076
1000,3.0,0.0,0.0,0.0,5.0,0.0,0.0,15.0,30.0,4.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1000,"[bc_049, bc_060, bc_066, bc_078, bc_081, bc_084]",6,bc_049-bc_060-bc_066-bc_078-bc_081-bc_084
10001,4.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,29.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10001,"[bc_045, bc_078, bc_084]",3,bc_045-bc_078-bc_084
10003,6.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,53.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10003,"[bc_002, bc_020, bc_033, bc_041, bc_058, bc_078]",6,bc_002-bc_020-bc_033-bc_041-bc_058-bc_078
10007,4.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,46.0,0.0,...,0.0,0.0,14.0,0.0,0.0,0.0,10007,"[bc_015, bc_017, bc_041, bc_045, bc_069, bc_07...",8,bc_015-bc_017-bc_041-bc_045-bc_069-bc_077-bc_0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9986,5.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,29.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9986,"[bc_013, bc_024, bc_052, bc_086]",4,bc_013-bc_024-bc_052-bc_086
9987,6.0,0.0,0.0,0.0,1.0,0.0,0.0,14.0,58.0,1.0,...,11.0,0.0,0.0,0.0,4.0,0.0,9987,"[bc_013, bc_015, bc_028, bc_047, bc_066, bc_08...",8,bc_013-bc_015-bc_028-bc_047-bc_066-bc_084-bc_0...
9989,2.0,0.0,0.0,0.0,2.0,0.0,0.0,9.0,23.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9989,"[bc_025, bc_063, bc_068]",3,bc_025-bc_063-bc_068


In [135]:
# plot comparision between the regular and proseg versions for each of the cell ids
# for cell_id in ['10','60','1000','2472','10000']:
for cell_id in ['1000','2472','10000']:
# for cell_id in ['60']:
    print(cell_id,mapping_dict[cell_id],df[df['cell_id']==cell_id]['called_barcodes'].values[0])
    sg_obj.plot_polygon_and_points(cell_id,gene_names=df[df['cell_id']==cell_id]['called_barcodes'].values[0],annotate=False,image_scale=5)
    sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=df[df['cell_id']==cell_id]['called_barcodes'].values[0],annotate=False,image_scale=3)

1000 9973 ['bc_049', 'bc_060', 'bc_066', 'bc_078', 'bc_081', 'bc_084']
2472 4005 ['bc_001', 'bc_022', 'bc_043', 'bc_060', 'bc_070']
10000 5003 ['bc_001', 'bc_008']


In [127]:
df['cell_id'][np.where(sg_obj.gdf.intersects(Polygon([(4388,601),(4388,602),(4387,601)])))[0][0]]


'10235'

In [131]:
sg_obj.gdf

,object_id,nucleus
0,35,"POLYGON ((9279.000 2.000, 9349.000 3.000, 9356..."
1,34,"POLYGON ((6437.000 2.000, 6520.000 2.000, 6527..."
2,29,"POLYGON ((24636.000 0.000, 24762.000 0.000, 24..."
3,11,"POLYGON ((11474.000 0.000, 11542.000 0.000, 11..."
4,15,"POLYGON ((14595.000 0.000, 14678.000 2.000, 14..."
...,...,...
16371,16364,"POLYGON ((19896.000 35571.000, 19914.000 35571..."
16372,16375,"POLYGON ((20380.000 35592.000, 20408.000 35596..."
16373,16361,"POLYGON ((22881.000 35565.000, 22904.000 35566..."
16374,16357,"POLYGON ((24819.000 35560.000, 24840.000 35560..."


Question: how does the probabilistic assignment in proseg work? Will it do a better job of assigning the spots to the correct cells and correct for these doublets?

In [136]:
for cell_id in ['10235']:
    print(cell_id,mapping_dict[cell_id],df[df['cell_id']==cell_id]['called_barcodes'].values[0])
    gene_names = df[df['cell_id']==cell_id]['called_barcodes'].values[0] + df[df['cell_id']=='60']['called_barcodes'].values[0]

    sg_obj.plot_polygon_and_points(cell_id,gene_names=gene_names,annotate=False,image_scale=5)
    sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=gene_names,annotate=False)

10235 10535 ['bc_001', 'bc_004', 'bc_038', 'bc_066']


In [227]:
def get_polygons_around_polygon(sg_obj,identifier,id_field='object_id',image_scale=5):

    polygon_gdf = sg_obj.gdf[sg_obj.gdf[id_field] == identifier]

    first_polygon_geometry = polygon_gdf.geometry.iloc[0]

    minx, miny, maxx, maxy = first_polygon_geometry.bounds
    dx = (maxx - minx) * 0.5 * image_scale
    dy = (maxy - miny) * 0.5 * image_scale
    expanded_bbox = box(minx - dx, miny - dy, maxx + dx, maxy + dy)

    other_polygons = sg_obj.gdf[sg_obj.gdf.geometry.intersects(expanded_bbox) & (sg_obj.gdf[id_field] != identifier)]

    return other_polygons

def get_polygon_barcodes(df,identifier,id_field='cell_id'):

    if len(df[df[id_field]==identifier]['called_barcodes'].values) > 0:

        return df[df[id_field]==identifier]['called_barcodes'].values[0]

    else:
        return []

def get_all_barcodes_in_region(df,identifiers,id_field='cell_id'):

    bc_list = []

    for identifier in identifiers:
        bc_list += get_polygon_barcodes(df,identifier,id_field=id_field)

    return bc_list



In [145]:
out = get_polygons_around_polygon(sg_obj,'10235',image_scale=5)

get_

In [156]:
for cell_id in ['10235']:
    # print(cell_id,mapping_dict[cell_id],df[df['cell_id']==cell_id]['called_barcodes'].values[0])

    region = get_polygons_around_polygon(sg_obj,cell_id,image_scale=5)

    gene_names = get_all_barcodes_in_region(df,region['object_id'].values)

    print(gene_names)

    # gene_names = df[df['cell_id']==cell_id]['called_barcodes'].values[0] + df[df['cell_id']=='60']['called_barcodes'].values[0]

    sg_obj.plot_polygon_and_points(cell_id,gene_names=gene_names,annotate=False,image_scale=5)
    # sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=gene_names,annotate=False)

['bc_041', 'bc_056', 'bc_060', 'bc_088', 'bc_092', 'bc_035', 'bc_083', 'bc_041', 'bc_056', 'bc_060', 'bc_088', 'bc_092', 'bc_004', 'bc_038', 'bc_066', 'bc_001', 'bc_004', 'bc_005', 'bc_009', 'bc_038', 'bc_066', 'bc_013', 'bc_024', 'bc_035', 'bc_052', 'bc_086', 'bc_001', 'bc_004', 'bc_066', 'bc_019', 'bc_040', 'bc_012', 'bc_045', 'bc_047', 'bc_057', 'bc_075', 'bc_076', 'bc_040', 'bc_012', 'bc_060']


In [157]:
get_all_barcodes_in_region(df,region['object_id'].values)

['bc_041',
 'bc_056',
 'bc_060',
 'bc_088',
 'bc_092',
 'bc_035',
 'bc_083',
 'bc_041',
 'bc_056',
 'bc_060',
 'bc_088',
 'bc_092',
 'bc_004',
 'bc_038',
 'bc_066',
 'bc_001',
 'bc_004',
 'bc_005',
 'bc_009',
 'bc_038',
 'bc_066',
 'bc_013',
 'bc_024',
 'bc_035',
 'bc_052',
 'bc_086',
 'bc_001',
 'bc_004',
 'bc_066',
 'bc_019',
 'bc_040',
 'bc_012',
 'bc_045',
 'bc_047',
 'bc_057',
 'bc_075',
 'bc_076',
 'bc_040',
 'bc_012',
 'bc_060']

In [275]:
sns.displot(df['IFIT2'],stat='density')
plt.yscale('log')

In [182]:
df[df['FN1']>100]

name,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096,cell_id,called_barcodes,n_called_barcodes,barcode_names
object_id,,,,,,,,,,,,,,,,,,,,,
10795,6.0,0.0,1.0,0.0,1.0,0.0,0.0,28.0,44.0,4.0,...,17.0,29.0,10.0,0.0,0.0,0.0,10795,"[bc_001, bc_031, bc_058, bc_086, bc_091, bc_09...",7,bc_001-bc_031-bc_058-bc_086-bc_091-bc_092-bc_093
14273,4.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14273,"[bc_035, bc_066]",2,bc_035-bc_066
16038,11.0,0.0,0.0,0.0,6.0,1.0,0.0,6.0,161.0,3.0,...,1.0,1.0,0.0,0.0,0.0,0.0,16038,[],0,
2681,8.0,0.0,0.0,0.0,5.0,0.0,0.0,40.0,156.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2681,"[bc_009, bc_014, bc_018, bc_041, bc_060, bc_06...",7,bc_009-bc_014-bc_018-bc_041-bc_060-bc_064-bc_086
2696,4.0,0.0,0.0,0.0,1.0,0.0,1.0,59.0,130.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2696,"[bc_018, bc_047, bc_064]",3,bc_018-bc_047-bc_064
2764,4.0,0.0,3.0,0.0,9.0,0.0,0.0,76.0,23.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2764,"[bc_011, bc_013, bc_020, bc_056, bc_080]",5,bc_011-bc_013-bc_020-bc_056-bc_080
3757,12.0,0.0,3.0,0.0,1.0,0.0,0.0,35.0,124.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,3757,"[bc_020, bc_030, bc_035, bc_037, bc_059, bc_06...",11,bc_020-bc_030-bc_035-bc_037-bc_059-bc_063-bc_0...
3864,9.0,0.0,0.0,0.0,9.0,0.0,0.0,14.0,51.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3864,[bc_060],1,bc_060
5818,19.0,0.0,0.0,0.0,28.0,0.0,0.0,26.0,201.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5818,[bc_019],1,bc_019


In [185]:
names,counts = np.unique(df[df['FN1']>60]['barcode_names'].values,return_counts=True)

names[np.where(counts>1)]

# names[np.argsort(counts)]

array(['', 'bc_004-bc_005-bc_013-bc_031-bc_088', 'bc_060'], dtype=object)

In [179]:
# df[df['barcode_names']=='bc_020-bc_058-bc_060-bc_071']


name,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096,cell_id,called_barcodes,n_called_barcodes,barcode_names
object_id,,,,,,,,,,,,,,,,,,,,,
3852,3.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,11.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3852,"[bc_004, bc_005, bc_013, bc_031, bc_088]",5,bc_004-bc_005-bc_013-bc_031-bc_088
3934,5.0,0.0,1.0,1.0,3.0,0.0,0.0,12.0,49.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3934,"[bc_004, bc_005, bc_013, bc_031, bc_088]",5,bc_004-bc_005-bc_013-bc_031-bc_088


In [188]:
scale = 8

for cell_id in ['3852']:
    # print(cell_id,mapping_dict[cell_id],df[df['cell_id']==cell_id]['called_barcodes'].values[0])

    region = get_polygons_around_polygon(sg_obj,cell_id,image_scale=scale)

    gene_names = get_all_barcodes_in_region(df,region['object_id'].values)

    print(gene_names)

    # gene_names = df[df['cell_id']==cell_id]['called_barcodes'].values[0] + df[df['cell_id']=='60']['called_barcodes'].values[0]

    sg_obj.plot_polygon_and_points(cell_id,gene_names=gene_names,annotate=False,image_scale=scale)
    sg_obj.plot_polygon_and_points(cell_id,gene_names=['FN1'],annotate=False,image_scale=scale)

    print(df[df['cell_id'].isin(region['object_id'].values)]['FN1'])
    # sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=gene_names,annotate=False)

['bc_013', 'bc_019', 'bc_048', 'bc_068', 'bc_087', 'bc_092', 'bc_013', 'bc_019', 'bc_039', 'bc_043', 'bc_044', 'bc_048', 'bc_066', 'bc_085', 'bc_003', 'bc_080', 'bc_013', 'bc_019', 'bc_039', 'bc_043', 'bc_044', 'bc_048', 'bc_066', 'bc_085', 'bc_004', 'bc_005', 'bc_013', 'bc_031', 'bc_088', 'bc_064', 'bc_013', 'bc_027', 'bc_001', 'bc_004', 'bc_013', 'bc_028', 'bc_044', 'bc_076', 'bc_088', 'bc_025', 'bc_031', 'bc_057', 'bc_076', 'bc_083', 'bc_087', 'bc_088', 'bc_002', 'bc_021', 'bc_025', 'bc_083', 'bc_084', 'bc_004', 'bc_005', 'bc_013', 'bc_076', 'bc_088']


object_id
3691     1.0
3736    29.0
3762    11.0
3809     4.0
3845    42.0
3899     1.0
3934    85.0
3946     1.0
4002     7.0
4032    12.0
4059     1.0
4063     5.0
4171     2.0
4175    28.0
Name: FN1, dtype: float32


In [197]:
adata_genes = sc.AnnData(df[gene_cols])

sc.pp.highly_variable_genes(adata_genes,n_top_genes=30,flavor='seurat_v3')

In [198]:
sc.pl.highly_variable_genes(adata_genes)

In [199]:
color_dict = {True:'r',False:'k'}

plt.figure()

plt.scatter(adata_genes.var['means'].values,adata_genes.var['variances_norm'].values,
            marker='.',s=5,color=[color_dict[x] for x in adata_genes.var['highly_variable']])

for gene in adata_genes.var.index:
    if adata_genes.var['highly_variable_rank'].loc[gene] < 10:
        plt.text(adata_genes.var['means'].loc[gene],adata_genes.var['variances_norm'].loc[gene],gene,fontsize=8)

In [200]:
for gene in adata_genes.var.index:
    if adata_genes.var['highly_variable_rank'].loc[gene] < 5:
        print(gene)
        sg_obj.show_gene_stats_plots(gene)



BGN
No filtering available or applied.
Statistics for gene BGN: count    16328.000000
mean         0.085987
std          0.726409
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         38.000000
Name: BGN, dtype: float64
Number of objects expressing BGN: 716 out of 16328 (4.39%)
FN1
No filtering available or applied.
Statistics for gene FN1: count    16328.000000
mean         4.646987
std          8.363742
min          0.000000
25%          1.000000
50%          2.000000
75%          5.000000
max        170.000000
Name: FN1, dtype: float64
Number of objects expressing FN1: 12748 out of 16328 (78.07%)
IFIT2
No filtering available or applied.


/Users/grantkinsler/miniforge3/envs/spatialgenomics-env2/lib/python3.9/site-packages/SGanalysis/SGobject.py:495: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(10, 10))


Statistics for gene IFIT2: count    16328.000000
mean         0.057937
std          0.535686
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         25.000000
Name: IFIT2, dtype: float64
Number of objects expressing IFIT2: 638 out of 16328 (3.91%)
IGFBP7
No filtering available or applied.
Statistics for gene IGFBP7: count    16328.000000
mean         0.291034
std          2.257929
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        116.000000
Name: IGFBP7, dtype: float64
Number of objects expressing IGFBP7: 2178 out of 16328 (13.34%)
MMP1
No filtering available or applied.
Statistics for gene MMP1: count    16328.000000
mean         0.084579
std          1.806872
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        119.000000
Name: MMP1, dtype: float64
Number of objects expressing MMP1: 298 out of 16328 (1.83%)


In [283]:
plt.figure()
sns.ecdfplot(df['IFIT2'].values,complementary=True)
plt.yscale('log')
plt.xlabel('IFIT2 spots per cell')
plt.ylabel('Fraction of cells with X spots or greater')

Text(0, 0.5, 'Fraction of cells with X spots or greater')

In [219]:
# gene_of_interest = 'IGFBP7'
gene_of_interest = 'IFIT2'

genes_to_plot = [gene_of_interest] + ['OASL']


names,counts = np.unique(df[df[gene_of_interest]>5]['barcode_names'].values,return_counts=True)

sisterhood_names = [x for x in names[np.where(counts>1)] if x.count('-')>1]

print(sisterhood_names)

['bc_013-bc_023-bc_035', 'bc_066-bc_075-bc_076']


In [241]:
scale = 5

# for sisterhood in sisterhood_names:

for sisterhood in sisterhood_names[:1]:

    print(sisterhood,df[df['barcode_names']==sisterhood]['cell_id'].values )

    cells_to_skip = []
    
    for cell_id in df[df['barcode_names']==sisterhood]['cell_id'].values:

        print(cell_id)
        # print(cell_id,mapping_dict[cell_id],df[df['cell_id']==cell_id]['called_barcodes'].values[0])

        if cell_id not in cells_to_skip:

            region = get_polygons_around_polygon(sg_obj,cell_id,image_scale=scale)

            for other_cell in df[df['barcode_names']==sisterhood]['cell_id'].values:
                if other_cell != cell_id:
                    if other_cell in region['object_id'].values:
                        cells_to_skip.append(other_cell)

            gene_names = get_all_barcodes_in_region(df,region['object_id'].values)

            print(gene_names)

            # gene_names = df[df['cell_id']==cell_id]['called_barcodes'].values[0] + df[df['cell_id']=='60']['called_barcodes'].values[0]

            sg_obj.plot_polygon_and_points(cell_id,gene_names=gene_names,annotate=False,image_scale=scale)
            sg_obj.plot_polygon_and_points(cell_id,gene_names=genes_to_plot,image_scale=scale,)
            sg_obj.plot_polygon_and_points(cell_id,gene_names=genes_to_plot,image_scale=2*scale,)

            print(df[df['cell_id'].isin(region['object_id'].values)][genes_to_plot])
            sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=gene_names,annotate=False,image_scale=scale,)
            sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=genes_to_plot,annotate=False,image_scale=scale,)

bc_013-bc_023-bc_035 ['4900' '4957' '4961' '5095']
4900
['bc_022', 'bc_013', 'bc_023', 'bc_035', 'bc_061', 'bc_073', 'bc_088', 'bc_013', 'bc_023', 'bc_035', 'bc_013', 'bc_023', 'bc_035', 'bc_013', 'bc_053', 'bc_086', 'bc_088', 'bc_061', 'bc_073', 'bc_088', 'bc_013', 'bc_022', 'bc_053', 'bc_020', 'bc_059']
name       IFIT2  OASL
object_id             
4821         0.0   2.0
4946         5.0   3.0
4957         8.0   0.0
4961         0.0   0.0
4986         0.0   0.0
5014         0.0   0.0
5053         0.0   0.0
5083         0.0   0.0
5090         0.0   0.0
4957
4961
5095
['bc_001', 'bc_066', 'bc_088', 'bc_052', 'bc_001', 'bc_066', 'bc_088', 'bc_001', 'bc_066', 'bc_088', 'bc_001', 'bc_023', 'bc_035', 'bc_066', 'bc_088', 'bc_013', 'bc_022', 'bc_023', 'bc_035', 'bc_001', 'bc_066', 'bc_088', 'bc_013', 'bc_022', 'bc_023', 'bc_035', 'bc_029', 'bc_029', 'bc_043', 'bc_062', 'bc_066', 'bc_073', 'bc_085', 'bc_029']
name       IFIT2  OASL
object_id             
5025         0.0   0.0
5051         0.

In [238]:
df[df['cell_id']=='5095']

name,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096,cell_id,called_barcodes,n_called_barcodes,barcode_names
object_id,,,,,,,,,,,,,,,,,,,,,
5095,4.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,22.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5095,"[bc_013, bc_023, bc_035]",3,bc_013-bc_023-bc_035


In [231]:
df[df['cell_id']=='4900']

name,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096,cell_id,called_barcodes,n_called_barcodes,barcode_names
object_id,,,,,,,,,,,,,,,,,,,,,
4900,2.0,1.0,0.0,0.0,5.0,0.0,0.0,6.0,25.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4900,"[bc_013, bc_023, bc_035]",3,bc_013-bc_023-bc_035


In [232]:
df[df['barcode_names']=='bc_013-bc_023-bc_035']

name,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096,cell_id,called_barcodes,n_called_barcodes,barcode_names
object_id,,,,,,,,,,,,,,,,,,,,,
4900,2.0,1.0,0.0,0.0,5.0,0.0,0.0,6.0,25.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4900,"[bc_013, bc_023, bc_035]",3,bc_013-bc_023-bc_035
4957,4.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,26.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4957,"[bc_013, bc_023, bc_035]",3,bc_013-bc_023-bc_035
4961,7.0,0.0,0.0,0.0,4.0,0.0,0.0,12.0,63.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4961,"[bc_013, bc_023, bc_035]",3,bc_013-bc_023-bc_035
5095,4.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,22.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5095,"[bc_013, bc_023, bc_035]",3,bc_013-bc_023-bc_035


In [256]:
# gene_of_interest = 'IGFBP7'
gene_of_interest = 'ITGA6'
gene_of_interest_cutoff = 20

# df[df['ITGA6']>20]

genes_to_plot = [gene_of_interest] + ['COL9A3','NGFR']


names,counts = np.unique(df[df[gene_of_interest]>gene_of_interest_cutoff ]['barcode_names'].values,return_counts=True)

sisterhood_names = [x for x in names[np.where(counts>1)] if x.count('-')>1]

print(sisterhood_names)

['bc_001-bc_012-bc_024-bc_026-bc_028-bc_056-bc_066-bc_075-bc_081', 'bc_024-bc_026-bc_048-bc_057-bc_063-bc_064-bc_066-bc_070-bc_074-bc_085-bc_088-bc_094']


In [257]:
scale = 5

# for sisterhood in sisterhood_names:

for sisterhood in sisterhood_names[:1]:

    print(sisterhood,df[df['barcode_names']==sisterhood]['cell_id'].values )

    cells_to_skip = []
    
    for cell_id in df[df['barcode_names']==sisterhood]['cell_id'].values:

        print(cell_id)
        # print(cell_id,mapping_dict[cell_id],df[df['cell_id']==cell_id]['called_barcodes'].values[0])

        if cell_id not in cells_to_skip:

            region = get_polygons_around_polygon(sg_obj,cell_id,image_scale=scale)

            for other_cell in df[df['barcode_names']==sisterhood]['cell_id'].values:
                if other_cell != cell_id:
                    if other_cell in region['object_id'].values:
                        cells_to_skip.append(other_cell)

            gene_names = get_all_barcodes_in_region(df,region['object_id'].values)

            print(gene_names)

            # gene_names = df[df['cell_id']==cell_id]['called_barcodes'].values[0] + df[df['cell_id']=='60']['called_barcodes'].values[0]

            sg_obj.plot_polygon_and_points(cell_id,gene_names=gene_names,annotate=False,image_scale=scale)
            sg_obj.plot_polygon_and_points(cell_id,gene_names=genes_to_plot,image_scale=scale,)
            sg_obj.plot_polygon_and_points(cell_id,gene_names=genes_to_plot,image_scale=2*scale,)

            print(df[df['cell_id'].isin(region['object_id'].values)][genes_to_plot])
            sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=gene_names,annotate=False,image_scale=scale,)
            # sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=genes_to_plot,annotate=False,image_scale=scale,)

bc_001-bc_012-bc_024-bc_026-bc_028-bc_056-bc_066-bc_075-bc_081 ['5012' '5073' '5108']
5012
['bc_020', 'bc_081', 'bc_082', 'bc_001', 'bc_024', 'bc_050', 'bc_060', 'bc_064', 'bc_084', 'bc_001', 'bc_014', 'bc_052', 'bc_057', 'bc_001', 'bc_002', 'bc_003', 'bc_020', 'bc_024', 'bc_027', 'bc_038', 'bc_050', 'bc_060', 'bc_064', 'bc_076', 'bc_084', 'bc_025', 'bc_081', 'bc_095', 'bc_025', 'bc_081', 'bc_095', 'bc_001', 'bc_012', 'bc_024', 'bc_026', 'bc_028', 'bc_056', 'bc_066', 'bc_075', 'bc_081', 'bc_001', 'bc_012', 'bc_024', 'bc_026', 'bc_028', 'bc_056', 'bc_066', 'bc_075', 'bc_081', 'bc_024', 'bc_028', 'bc_066', 'bc_075', 'bc_081', 'bc_090', 'bc_029', 'bc_043', 'bc_066', 'bc_073', 'bc_084', 'bc_085', 'bc_090']


name       ITGA6  COL9A3  NGFR
object_id                     
4749         6.0     2.0   0.0
4779         3.0     2.0   0.0
4811         1.0     0.0   0.0
4830         9.0    13.0   0.0
4877         6.0     5.0   0.0
5013         4.0     0.0   0.0
5073        29.0    19.0   0.0
5100         8.0     2.0   0.0
5108        13.0    15.0   0.0
5223         8.0     9.0   0.0
5250         5.0     8.0   0.0
5278         0.0     3.0   0.0
5304         1.0     8.0   0.0
5073
5108


In [284]:
sns.displot(df['MLANA'])

In [287]:
df[df['MLANA']>20]

name,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096,cell_id,called_barcodes,n_called_barcodes,barcode_names
object_id,,,,,,,,,,,,,,,,,,,,,
10045,11.0,0.0,0.0,0.0,23.0,0.0,0.0,26.0,92.0,14.0,...,0.0,0.0,0.0,0.0,0.0,1.0,10045,"[bc_009, bc_063, bc_065]",3,bc_009-bc_063-bc_065
10093,14.0,0.0,0.0,0.0,4.0,0.0,0.0,14.0,90.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10093,[bc_071],1,bc_071
10133,25.0,0.0,0.0,1.0,2.0,0.0,0.0,12.0,271.0,1.0,...,0.0,52.0,0.0,0.0,0.0,0.0,10133,"[bc_001, bc_014, bc_015, bc_024, bc_043, bc_04...",10,bc_001-bc_014-bc_015-bc_024-bc_043-bc_045-bc_0...
10497,11.0,0.0,0.0,1.0,11.0,0.0,0.0,14.0,240.0,3.0,...,0.0,0.0,1.0,0.0,0.0,0.0,10497,"[bc_004, bc_043, bc_070, bc_083]",4,bc_004-bc_043-bc_070-bc_083
10577,26.0,0.0,0.0,1.0,12.0,0.0,0.0,20.0,162.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10577,[bc_003],1,bc_003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8707,16.0,0.0,0.0,0.0,25.0,0.0,0.0,46.0,208.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8707,[bc_003],1,bc_003
8788,15.0,0.0,0.0,0.0,12.0,0.0,0.0,9.0,106.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8788,"[bc_063, bc_074, bc_075, bc_084]",4,bc_063-bc_074-bc_075-bc_084
8825,40.0,0.0,1.0,1.0,26.0,0.0,2.0,29.0,317.0,2.0,...,0.0,0.0,1.0,4.0,0.0,0.0,8825,"[bc_001, bc_013, bc_024, bc_040, bc_057, bc_05...",12,bc_001-bc_013-bc_024-bc_040-bc_057-bc_059-bc_0...


In [288]:
# gene_of_interest = 'IGFBP7'
gene_of_interest = 'MLANA'
gene_of_interest_cutoff = 20

# df[df['ITGA6']>20]

# genes_to_plot = [gene_of_interest] + ['COL9A3','NGFR']
genes_to_plot = [gene_of_interest]


names,counts = np.unique(df[df[gene_of_interest]>gene_of_interest_cutoff ]['barcode_names'].values,return_counts=True)

sisterhood_names = [x for x in names[np.where(counts>1)] if x.count('-')>1]

print(sisterhood_names)

['bc_001-bc_006-bc_011-bc_019-bc_026-bc_031-bc_041-bc_053-bc_056-bc_059-bc_060-bc_063-bc_064-bc_068-bc_070-bc_081-bc_086-bc_087-bc_092', 'bc_004-bc_043-bc_070-bc_083', 'bc_015-bc_043-bc_044-bc_058-bc_068']


In [290]:
scale = 5

# for sisterhood in sisterhood_names:

for sisterhood in sisterhood_names[1:]:

    print(sisterhood,df[df['barcode_names']==sisterhood]['cell_id'].values )

    cells_to_skip = []
    
    for cell_id in df[df['barcode_names']==sisterhood]['cell_id'].values:

        print(cell_id)
        # print(cell_id,mapping_dict[cell_id],df[df['cell_id']==cell_id]['called_barcodes'].values[0])

        if cell_id not in cells_to_skip:

            region = get_polygons_around_polygon(sg_obj,cell_id,image_scale=scale)

            for other_cell in df[df['barcode_names']==sisterhood]['cell_id'].values:
                if other_cell != cell_id:
                    if other_cell in region['object_id'].values:
                        cells_to_skip.append(other_cell)

            gene_names = get_all_barcodes_in_region(df,region['object_id'].values)

            print(gene_names)

            # gene_names = df[df['cell_id']==cell_id]['called_barcodes'].values[0] + df[df['cell_id']=='60']['called_barcodes'].values[0]

            sg_obj.plot_polygon_and_points(cell_id,gene_names=gene_names,annotate=False,image_scale=scale)
            sg_obj.plot_polygon_and_points(cell_id,gene_names=genes_to_plot,image_scale=scale,)
            # sg_obj.plot_polygon_and_points(cell_id,gene_names=genes_to_plot,image_scale=2*scale,)

            print(df[df['cell_id'].isin(region['object_id'].values)][genes_to_plot])
            sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=gene_names,annotate=False,image_scale=scale,)
            # sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=genes_to_plot,annotate=False,image_scale=scale,)

bc_004-bc_043-bc_070-bc_083 ['10497' '10719']
10497
['bc_033', 'bc_066', 'bc_022', 'bc_033', 'bc_059', 'bc_083', 'bc_091', 'bc_091', 'bc_059', 'bc_083', 'bc_091', 'bc_059', 'bc_083', 'bc_004', 'bc_043', 'bc_070', 'bc_083', 'bc_020', 'bc_044']
name       MLANA
object_id       
10252        1.0
10288        7.0
10347        4.0
10369        3.0
10382        4.0
10423        6.0
10486        5.0
10514        3.0
10521       13.0
10590        4.0
10615        4.0
10652       11.0
10683        5.0
10719       23.0
10747        3.0
10758        3.0
10767        5.0
10800        4.0
10807        0.0
10833        6.0
10719
bc_015-bc_043-bc_044-bc_058-bc_068 ['13195' '13426']
13195
['bc_057', 'bc_014', 'bc_020', 'bc_033', 'bc_057', 'bc_057', 'bc_074', 'bc_014', 'bc_058', 'bc_060', 'bc_014', 'bc_033', 'bc_057', 'bc_014', 'bc_008', 'bc_024', 'bc_037', 'bc_040', 'bc_041', 'bc_052', 'bc_067', 'bc_075', 'bc_077', 'bc_078', 'bc_039', 'bc_043', 'bc_066', 'bc_078', 'bc_012', 'bc_060', 'bc_080', 'bc_014

In [297]:
df[df['EGFR']>2]

name,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096,cell_id,called_barcodes,n_called_barcodes,barcode_names
object_id,,,,,,,,,,,,,,,,,,,,,
10190,5.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,67.0,0.0,...,0.0,0.0,0.0,0.0,23.0,0.0,10190,"[bc_004, bc_022, bc_033, bc_041, bc_052, bc_095]",6,bc_004-bc_022-bc_033-bc_041-bc_052-bc_095
102,4.0,0.0,0.0,0.0,6.0,0.0,0.0,5.0,33.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,102,[],0,
10437,5.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,38.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10437,[bc_019],1,bc_019
10539,5.0,1.0,0.0,0.0,4.0,0.0,0.0,8.0,13.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10539,[bc_050],1,bc_050
11000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,82.0,3.0,...,0.0,2.0,0.0,0.0,0.0,0.0,11000,"[bc_001, bc_060, bc_073]",3,bc_001-bc_060-bc_073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,15.0,0.0,0.0,0.0,9.0,0.0,1.0,12.0,133.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,975,[],0,
9875,9.0,0.0,0.0,0.0,7.0,0.0,0.0,12.0,91.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9875,"[bc_003, bc_025, bc_085]",3,bc_003-bc_025-bc_085
9884,6.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,22.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9884,"[bc_066, bc_084]",2,bc_066-bc_084


In [298]:
# gene_of_interest = 'IGFBP7'
gene_of_interest = 'EGFR'
gene_of_interest_cutoff = 2

# df[df['ITGA6']>20]

# genes_to_plot = [gene_of_interest] + ['COL9A3','NGFR']
genes_to_plot = [gene_of_interest]


names,counts = np.unique(df[df[gene_of_interest]>gene_of_interest_cutoff ]['barcode_names'].values,return_counts=True)

sisterhood_names = [x for x in names[np.where(counts>1)] if x.count('-')>1]

print(sisterhood_names)

['bc_001-bc_018-bc_028-bc_058', 'bc_015-bc_022-bc_035-bc_041']


In [301]:
scale = 5

# for sisterhood in sisterhood_names:

for sisterhood in sisterhood_names:

    print(sisterhood,df[df['barcode_names']==sisterhood]['cell_id'].values )

    cells_to_skip = []
    
    for cell_id in df[df['barcode_names']==sisterhood]['cell_id'].values:

        print(cell_id)
        # print(cell_id,mapping_dict[cell_id],df[df['cell_id']==cell_id]['called_barcodes'].values[0])

        if cell_id not in cells_to_skip:

            region = get_polygons_around_polygon(sg_obj,cell_id,image_scale=scale)

            for other_cell in df[df['barcode_names']==sisterhood]['cell_id'].values:
                if other_cell != cell_id:
                    if other_cell in region['object_id'].values:
                        cells_to_skip.append(other_cell)

            gene_names = get_all_barcodes_in_region(df,region['object_id'].values)

            print(gene_names)

            # gene_names = df[df['cell_id']==cell_id]['called_barcodes'].values[0] + df[df['cell_id']=='60']['called_barcodes'].values[0]

            sg_obj.plot_polygon_and_points(cell_id,gene_names=gene_names,annotate=False,image_scale=scale)
            sg_obj.plot_polygon_and_points(cell_id,gene_names=genes_to_plot,image_scale=scale,)
            # sg_obj.plot_polygon_and_points(cell_id,gene_names=genes_to_plot,image_scale=2*scale,)

            print(df[df['cell_id'].isin(region['object_id'].values)][genes_to_plot])
            sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=gene_names,annotate=False,image_scale=scale,)
            # sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=genes_to_plot,annotate=False,image_scale=scale,)

bc_001-bc_018-bc_028-bc_058 ['624' '827' '917']
624
['bc_039', 'bc_074', 'bc_063', 'bc_039', 'bc_057', 'bc_086', 'bc_039', 'bc_074', 'bc_001', 'bc_039', 'bc_074', 'bc_022', 'bc_039', 'bc_057', 'bc_086', 'bc_057']
name       EGFR
object_id      
426         7.0
439         0.0
486         2.0
552         0.0
572         0.0
593         1.0
617         0.0
628         2.0
687         0.0
701         0.0
731         1.0
827
['bc_045', 'bc_082', 'bc_039', 'bc_074', 'bc_018', 'bc_028', 'bc_066', 'bc_033', 'bc_035', 'bc_001', 'bc_018', 'bc_028', 'bc_058', 'bc_074', 'bc_083', 'bc_075', 'bc_074', 'bc_083', 'bc_001', 'bc_015', 'bc_081']
name       EGFR
object_id      
699         0.0
706         0.0
731         1.0
804         1.0
903         2.0
906         0.0
917         1.0
954         1.0
958         1.0
968         3.0
978         1.0
917
bc_015-bc_022-bc_035-bc_041 ['560' '603']
560
['bc_025', 'bc_031', 'bc_035', 'bc_015', 'bc_039', 'bc_062', 'bc_073', 'bc_092', 'bc_090', 'bc_015', 'bc_0

In [304]:
# gene_of_interest = 'IGFBP7'
gene_of_interest = 'SOX10'
gene_of_interest_cutoff = 45

# df[df['ITGA6']>20]

# genes_to_plot = [gene_of_interest] + ['COL9A3','NGFR']
genes_to_plot = [gene_of_interest]


names,counts = np.unique(df[df[gene_of_interest]>gene_of_interest_cutoff ]['barcode_names'].values,return_counts=True)

sisterhood_names = [x for x in names[np.where(counts>1)] if x.count('-')>1]

print(sisterhood_names)

['bc_001-bc_015-bc_081', 'bc_004-bc_010-bc_017-bc_018-bc_027-bc_043-bc_048-bc_066-bc_085-bc_087-bc_095', 'bc_015-bc_043-bc_044-bc_058-bc_068']


In [305]:
scale = 5

# for sisterhood in sisterhood_names:

for sisterhood in sisterhood_names:

    print(sisterhood,df[df['barcode_names']==sisterhood]['cell_id'].values )

    cells_to_skip = []
    
    for cell_id in df[df['barcode_names']==sisterhood]['cell_id'].values:

        print(cell_id)
        # print(cell_id,mapping_dict[cell_id],df[df['cell_id']==cell_id]['called_barcodes'].values[0])

        if cell_id not in cells_to_skip:

            region = get_polygons_around_polygon(sg_obj,cell_id,image_scale=scale)

            for other_cell in df[df['barcode_names']==sisterhood]['cell_id'].values:
                if other_cell != cell_id:
                    if other_cell in region['object_id'].values:
                        cells_to_skip.append(other_cell)

            gene_names = get_all_barcodes_in_region(df,region['object_id'].values)

            print(gene_names)

            # gene_names = df[df['cell_id']==cell_id]['called_barcodes'].values[0] + df[df['cell_id']=='60']['called_barcodes'].values[0]

            sg_obj.plot_polygon_and_points(cell_id,gene_names=gene_names,annotate=False,image_scale=scale)
            sg_obj.plot_polygon_and_points(cell_id,gene_names=genes_to_plot,image_scale=scale,)
            # sg_obj.plot_polygon_and_points(cell_id,gene_names=genes_to_plot,image_scale=2*scale,)

            print(df[df['cell_id'].isin(region['object_id'].values)][genes_to_plot])
            sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=gene_names,annotate=False,image_scale=scale,)
            # sg_obj_only_bcs.plot_polygon_and_points(mapping_dict[cell_id],gene_names=genes_to_plot,annotate=False,image_scale=scale,)

bc_001-bc_015-bc_081 ['958' '997']
958
['bc_045', 'bc_083', 'bc_009', 'bc_047', 'bc_056', 'bc_064', 'bc_068', 'bc_079', 'bc_091', 'bc_009', 'bc_045', 'bc_064', 'bc_068', 'bc_075', 'bc_001', 'bc_018', 'bc_028', 'bc_058', 'bc_059', 'bc_075', 'bc_075', 'bc_001', 'bc_018', 'bc_028', 'bc_058', 'bc_075', 'bc_074', 'bc_083', 'bc_056', 'bc_059', 'bc_068', 'bc_079', 'bc_084', 'bc_075', 'bc_074', 'bc_083', 'bc_075', 'bc_024', 'bc_045', 'bc_055', 'bc_065', 'bc_081', 'bc_083', 'bc_001', 'bc_015', 'bc_081', 'bc_075', 'bc_033', 'bc_039', 'bc_056', 'bc_063', 'bc_075', 'bc_001', 'bc_033', 'bc_056', 'bc_068', 'bc_074', 'bc_079', 'bc_033', 'bc_056', 'bc_075', 'bc_082', 'bc_039', 'bc_056', 'bc_075', 'bc_024', 'bc_045', 'bc_055', 'bc_081', 'bc_083', 'bc_033', 'bc_039', 'bc_056', 'bc_063', 'bc_075', 'bc_082', 'bc_060', 'bc_068', 'bc_060', 'bc_068', 'bc_033', 'bc_039', 'bc_056', 'bc_082', 'bc_067', 'bc_060', 'bc_056', 'bc_068', 'bc_074', 'bc_079', 'bc_084', 'bc_033', 'bc_039', 'bc_056', 'bc_075', 'bc_082']


In [308]:
df[df['VEGFA']>2]

name,AKT1,AXL,BGN,BIRC3,BIRC5,BMP2,BMP4,CALD1,CAPG,CCNA2,...,bc_091,bc_092,bc_093,bc_094,bc_095,bc_096,cell_id,called_barcodes,n_called_barcodes,barcode_names
object_id,,,,,,,,,,,,,,,,,,,,,
10011,12.0,0.0,1.0,0.0,14.0,0.0,0.0,17.0,128.0,7.0,...,34.0,0.0,0.0,0.0,0.0,0.0,10011,"[bc_043, bc_063, bc_071, bc_091]",4,bc_043-bc_063-bc_071-bc_091
10015,2.0,0.0,0.0,0.0,3.0,1.0,0.0,3.0,22.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,10015,"[bc_007, bc_050]",2,bc_007-bc_050
10027,3.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,27.0,1.0,...,0.0,25.0,0.0,0.0,0.0,0.0,10027,"[bc_041, bc_056, bc_060, bc_088, bc_092]",5,bc_041-bc_056-bc_060-bc_088-bc_092
10042,0.0,0.0,6.0,0.0,0.0,0.0,0.0,4.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10042,[bc_060],1,bc_060
10061,7.0,0.0,0.0,0.0,5.0,0.0,0.0,11.0,49.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10061,"[bc_004, bc_033, bc_038, bc_066]",4,bc_004-bc_033-bc_038-bc_066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,7.0,0.0,0.0,0.0,3.0,0.0,0.0,11.0,27.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,996,[bc_060],1,bc_060
9977,7.0,0.0,0.0,0.0,6.0,0.0,0.0,10.0,40.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9977,"[bc_009, bc_039, bc_052]",3,bc_009-bc_039-bc_052
9984,10.0,0.0,0.0,0.0,3.0,0.0,0.0,19.0,155.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9984,"[bc_001, bc_022, bc_033, bc_076]",4,bc_001-bc_022-bc_033-bc_076


In [307]:
sns.displot(df['VEGFA'])